# Проект по SQL

Компания купила крупный сервис для чтения книг по подписке. Ваша задача проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Цели проекта:

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (это исключает брошюры);
- Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Загрузка данных и подготовка их к анализу

In [1]:
# импортируем библиотеки 
import pandas as pd 
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры 
db_config = {'user': 'praktikum_student', # имя пользователя 
                     'pwd': 'Sdf4$2;d-d30pp', # пароль 
                     'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
                     'port': 6432, # порт подключения 
                     'db': 'data-analyst-final-project-db'} # название базы данных 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db']) 

# сохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Формируем sql запрос
query = ''' select
                book_id,
                author_id,
                title,
                num_pages,
                publication_date,
                publisher_id
            from books
            limit 5
        '''

In [4]:
# Выполняем запрос и сохраняем результат в books
books = pd.io.sql.read_sql(query, con = engine)

In [5]:
# Выведем первые 5 строк таблицы
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [6]:
# Формируем sql запрос
query = ''' select
                author_id,
                author
            from authors
            limit 5
        '''

In [7]:
# Выполняем запрос и сохраняем результат в authors
authors = pd.io.sql.read_sql(query, con = engine)

In [8]:
# Выведем первые 5 строк таблицы
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [9]:
# Формируем sql запрос
query = ''' select
                publisher_id,
                publisher
            from publishers
            limit 5
        '''

In [10]:
# Выполняем запрос и сохраняем результат в publishers
publishers = pd.io.sql.read_sql(query, con = engine)

In [11]:
# Выведем первые 5 строк таблицы
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [12]:
# Формируем sql запрос
query = ''' select 
                rating_id, 
                book_id, 
                username, 
                rating
            from ratings
            limit 5
        '''

In [13]:
# Выполняем запрос и сохраняем результат в ratings
ratings = pd.io.sql.read_sql(query, con = engine)

In [14]:
# Выведем первые 5 строк таблицы
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [15]:
# Формируем sql запрос
query = ''' select 
                review_id, 
                book_id, 
                username, 
                text
            from reviews
            limit 5
        '''

In [16]:
# Выполняем запрос и сохраняем результат в reviews
reviews = pd.io.sql.read_sql(query, con = engine)

In [17]:
# Выведем первые 5 строк таблицы
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## SQL - запросы

### Cколько книг вышло после 1 января 2000 года?

In [18]:
# Формируем sql запрос
query = ''' select 
                count(title) as cnt_title
            from books
            where cast(publication_date as date) > '2000-01-01'
        '''

In [19]:
# Выполняем запрос и сохраняем результат в books_01_2000
df = pd.io.sql.read_sql(query, con = engine)
df

,cnt_title
0,819


#### Вывод

После 1 января вышло 819 книг.

### Какое количество обзоров и какая средняя оценка для каждой из книг?

In [20]:
# Формируем sql запрос
query = ''' select 
                books.book_id as book, 
                avg(ratings.rating) as reviews_mean, 
                count(reviews.book_id) as reviews_amount
            from books
                left join ratings
                    on books.book_id = ratings.book_id 
                        left join reviews
                            on books.book_id = reviews.book_id
            group by book
            order by reviews_mean
        '''

In [21]:
df_1 = pd.io.sql.read_sql(query, con = engine)
df_1

,book,reviews_mean,reviews_amount
0,303,1.50,4
1,202,2.00,9
2,371,2.00,4
3,316,2.00,4
4,915,2.25,12
...,...,...,...
995,182,5.00,4
996,599,5.00,2
997,62,5.00,4
998,913,5.00,4


#### Вывод

Я вывела таблицу с id книги и её средней оценкой и количеством отзывов с сортировкой по средней оценке

### Какое издательство, которое выпустило наибольшее число книг толще 50 страниц (это исключает брошюры)?

In [22]:
# Формируем sql запрос
query = ''' select
                publishers.publisher as publishing_house, 
                count(title) as cnt_title
            from books 
                left join publishers 
                    on books.publisher_id = publishers.publisher_id
            where num_pages > '50'
            group by publishing_house
            order by cnt_title
        '''

In [23]:
df_2 = pd.io.sql.read_sql(query, con = engine)
df_2

,publishing_house,cnt_title
0,Alfred A. Knopf Borzoi Books,1
1,Arthur A. Levine Books / Scholastic Inc.,1
2,Dante University of America Press,1
3,Red Fox,1
4,Citadel,1
...,...,...
329,Ballantine Books,19
330,Penguin Classics,24
331,Grand Central Publishing,25
332,Vintage,31


#### Вывод

Издательство Penguin Books выпустило больше всего книг толще 50 страниц, а именно 42 книги.

### Какой автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками?

In [24]:
# Формируем sql запрос
query = ''' select
                aql.author as author,
                avg(ratings.rating) as ratings_mean
            from(
                select
                    ratings.book_id,
                    authors.author as author, 
                    count(ratings.rating) as ratings_amount 
                from books 
                    left join ratings 
                        on books.book_id = ratings.book_id
                            left join authors 
                                on books.author_id = authors.author_id
                group by 
                    author,
                    ratings.book_id
                having count(ratings.rating) > 50
                order by ratings_amount) as aql
                left join ratings 
                    on aql.book_id = ratings.book_id
            group by author
            order by ratings_mean
        '''

In [25]:
df_3 = pd.io.sql.read_sql(query, con = engine)
df_3

,author,ratings_mean
0,John Steinbeck,3.622951
1,Stephenie Meyer,3.662500
2,George Orwell/Boris Grabnar/Peter Škerl,3.729730
3,Dan Brown,3.741259
4,Lois Lowry,3.750000
5,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
6,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
7,J.D. Salinger,3.825581
8,William Golding,3.901408
9,Rick Riordan,4.080645


#### Вывод

Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку среди книг с 50ю и более оценками, а именно 4,287097 балла.

### Какое среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [26]:
# Формируем sql запрос
query = ''' select 
                avg(reviews_amount)
            from(
                select 
                    distinct abs.name,
                    count(reviews.text) as reviews_amount
                from(
                    select 
                        username as name,
                        count(rating) as ratings_amount
                    from ratings
                    group by name
                    having count(ratings.rating) > 50) as abs
                        left join reviews 
                            on abs.name = reviews.username
                        group by abs.name) as sba
        '''

In [27]:
df_4 = pd.io.sql.read_sql(query, con = engine)
df_4

,avg
0,24.333333


#### Вывод

В среднем пользователи, которые поставили больше 50 оценок, оставляют 24-25 обзоров (24,33)

## Вывод

- После 1 января вышло 819 книг;
- Каждая книга имеет среднюю оценку от 1,5 до 5 баллов, и от 0 до 1120 отзывов. В зависимости от книги;
- Издательство Penguin Books выпустило больше всего книг толще 50 страниц, а именно 42 книги;
- Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку среди книг с 50ю и более оценками, а именно 4,287097 балла;
- В среднем пользователи, которые поставили больше 50 оценок, оставляют 24-25 обзоров (24,33).